Scraping International Player Stats

In [1]:
import pandas as pd
import numpy as np
import requests
import lxml.html as lh
from bs4 import BeautifulSoup

In [4]:
file = 'C:\\Users\\Chung-Tseng Wang\\Desktop\\NBA-Rookie-Stat-Predictions\\data\\processed\\final.csv'
#file = '//Users//chung-tsengwang//Desktop//Github Repo//Predicting-NBA-Rookie-Stats//data//processed//final.csv'
nba = pd.read_csv(file, index_col = 'Player')
nba.head()

,Age,NBA_3PP,NBA_AST,NBA_BLK,NBA_FGP,NBA_FTP,NBA_G,NBA_PTS,NBA_STL,NBA_TOV,...,c_AST,c_BLK,c_FGP,c_FTP,c_G,c_PTS,c_STL,c_TOV,c_TRB,year
Player,,,,,,,,,,,,,,,,,,,,,
A.J. Hammons,24,0.500,0.2,0.590909,0.405,0.450,22,2.2,0.045455,0.454545,...,0.8,2.6,0.539,0.690,132.0,12.1,0.4,2.1,7.0,2017
A.J. Price,23,0.316,2.2,0.034483,0.380,0.742,261,5.8,0.478927,0.877395,...,4.7,0.1,0.413,0.722,99.0,13.0,1.1,2.4,3.4,2010
Aaron Gordon,19,0.321,2.2,0.639296,0.450,0.707,341,12.5,0.765396,1.354839,...,2.0,1.0,0.495,0.422,38.0,12.4,0.9,1.4,8.0,2015
Aaron Harrison,21,0.218,0.5,0.057143,0.261,0.613,35,2.3,0.428571,0.200000,...,1.6,0.2,0.410,0.787,79.0,12.4,1.1,1.3,2.8,2016
Aaron Holiday,22,0.339,1.7,0.260000,0.401,0.820,50,5.9,0.420000,0.800000,...,4.7,0.2,0.450,0.795,101.0,14.3,1.2,3.0,3.2,2019


In [5]:
players_list = nba[nba['School'].isnull()].index.values
len(players_list)

160

In [6]:
players_list

array(['Alex Abrines', 'Alexey Shved', 'Alexis Ajinca', 'Andrew White',
       'Anfernee Simons', 'Angel Delgado', 'Ante Zizic', 'Axel Toupane',
       'Bam Adebayo', 'Bernard James', 'Bismack Biyombo',
       'Boban Marjanovic', 'Bogdan Bogdanovic', 'Bojan Bogdanovic',
       'Bruno Caboclo', 'Bryce Dejean-Jones', 'Byron Mullens',
       'Cedi Osman', 'Christian Eyenga', 'Clint Capela',
       'Cristiano Felicio', 'Dairis Bertans', 'Damien Inglis',
       'Damjan Rudez', 'Daniel Theis', 'Danilo Gallinari', 'Dante Exum',
       'Dario Saric', 'David Andersen', 'Davis Bertans',
       'Dennis Schroder', 'Devyn Marble', 'Donatas Motiejunas',
       'Donte Greene', 'Dragan Bender', 'Dzanan Musa', 'Edy Tavares',
       'Elie Okobo', 'Emanuel Terry', 'Emmanuel Mudiay', 'Enes Kanter',
       'Eric Dawson', 'Eugene Jeter', 'Evan Fournier', 'Frank Ntilikina',
       'Fred VanVleet', 'Furkan Aldemir', 'Furkan Korkmaz',
       'Garret Siler', 'Georgios Papagiannis', 'Gerald Henderson',
       'G

In [42]:
data = []
for player in players_list:
    url_name = player
    player = player.lower()
    player = player.replace('.','')
    player = player.replace("'",'')
    player = player.replace(' ','-')
    website_url = requests.get('https://www.basketball-reference.com/international/players/'+ player +'-1.html')
    soup = BeautifulSoup(website_url.content,'lxml')
    table = soup.find('table',{'id':'per_game-clu-reg'})
    if table is not None:
        data = data + [url_name]
        stats = table.findAll('td')
        elements = []
        for n in stats:
            elements.append(n.get_text())
        data = data + elements[-22:]

In [40]:
data2 = []
for player in players_list:
    url_name = player
    player = player.lower()
    player = player.replace('.','')
    player = player.replace("'",'')
    player = player.replace(' ','-')
    website_url = requests.get('https://www.basketball-reference.com/international/players/'+ player +'-1.html')
    soup = BeautifulSoup(website_url.content,'lxml')
    #Position 
    div = soup.find_all('div',{'class' : 'nothumb'})
    for x in div:
        data2 = data2 + [url_name]
        ad = x.find_all('p')
        position = (ad[0].text.replace('Position:', "").strip())
    data2 = data2 + [position]

In [43]:
transformed = np.reshape(np.array(data),(-1,23))

In [44]:
transformed2 = np.reshape(np.array(data2),(-1,2))

In [48]:
#To get column headers
headers = ['Player','MP','FG','FGA','FG%','3P','3PA','3P%','2P','2PA','2P%','FT','FTA','FT%','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS']
inter = pd.DataFrame(transformed, columns = headers)
inter['3P%'] = inter['3P%'].replace('',0)
inter['FT%'] = inter['FT%'].replace('',0)
inter['2P%'] = inter['3P%'].replace('',0)
inter['eFGP'] = 0
inter['PER'] = 0
inter['WS'] = 0
inter.head()

,Player,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,...,TRB,AST,STL,BLK,TOV,PF,PTS,eFGP,PER,WS
0,Alex Abrines,17.2,2.4,5.2,.463,1.4,3.3,.423,1.0,2.0,...,1.7,0.8,0.6,0.3,0.6,1.8,7.3,0,0,0
1,Alexis Ajinca,19.9,4.5,8.2,.544,0.2,0.6,.293,4.3,7.6,...,5.2,1.0,0.7,1.4,2.2,2.8,11.7,0,0,0
2,Axel Toupane,17.1,2.1,4.6,.464,0.7,2.0,.353,1.4,2.6,...,2.2,0.9,0.8,0.1,0.7,1.9,5.7,0,0,0
3,Bismack Biyombo,17.1,2.3,4.1,.552,0.0,0.0,0,2.3,4.1,...,5.1,0.3,0.4,2.3,1.7,2.0,6.4,0,0,0
4,Bojan Bogdanovic,5.6,0.1,1.0,.143,0.1,1.0,.143,0.0,0.0,...,0.4,0.1,0.3,0.0,0.4,1.3,1.0,0,0,0


In [49]:
headers = ['Player','Position']
position = pd.DataFrame(transformed2, columns = headers)
inter = inter.merge(position, how = 'left', on = 'Player')
inter.head()

,Player,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,...,AST,STL,BLK,TOV,PF,PTS,eFGP,PER,WS,Position
0,Alex Abrines,17.2,2.4,5.2,.463,1.4,3.3,.423,1.0,2.0,...,0.8,0.6,0.3,0.6,1.8,7.3,0,0,0,Forward
1,Alexis Ajinca,19.9,4.5,8.2,.544,0.2,0.6,.293,4.3,7.6,...,1.0,0.7,1.4,2.2,2.8,11.7,0,0,0,Center
2,Axel Toupane,17.1,2.1,4.6,.464,0.7,2.0,.353,1.4,2.6,...,0.9,0.8,0.1,0.7,1.9,5.7,0,0,0,NaN
3,Bismack Biyombo,17.1,2.3,4.1,.552,0.0,0.0,0,2.3,4.1,...,0.3,0.4,2.3,1.7,2.0,6.4,0,0,0,Forward
4,Bojan Bogdanovic,5.6,0.1,1.0,.143,0.1,1.0,.143,0.0,0.0,...,0.1,0.3,0.0,0.4,1.3,1.0,0,0,0,"6-7, 216lb (201cm, 97kg)"


In [50]:
inter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71 entries, 0 to 70
Data columns (total 27 columns):
Player      71 non-null object
MP          71 non-null object
FG          71 non-null object
FGA         71 non-null object
FG%         71 non-null object
3P          71 non-null object
3PA         71 non-null object
3P%         71 non-null object
2P          71 non-null object
2PA         71 non-null object
2P%         71 non-null object
FT          71 non-null object
FTA         71 non-null object
FT%         71 non-null object
ORB         71 non-null object
DRB         71 non-null object
TRB         71 non-null object
AST         71 non-null object
STL         71 non-null object
BLK         71 non-null object
TOV         71 non-null object
PF          71 non-null object
PTS         71 non-null object
eFGP        71 non-null int64
PER         71 non-null int64
WS          71 non-null int64
Position    29 non-null object
dtypes: int64(3), object(24)
memory usage: 15.5+ KB


In [11]:
inter.describe()

,eFGP,PER,WS
count,160.0,160.0,160.0
mean,0.0,0.0,0.0
std,0.0,0.0,0.0
min,0.0,0.0,0.0
25%,0.0,0.0,0.0
50%,0.0,0.0,0.0
75%,0.0,0.0,0.0
max,0.0,0.0,0.0


In [51]:
inter.columns

Index(['Player', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA',
       '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'PTS', 'eFGP', 'PER', 'WS', 'Position'],
      dtype='object')

In [52]:
list_ = ['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA',
       '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'PTS', 'eFGP', 'PER', 'WS']
for n in list_:
    inter[n] = inter[n].astype(float)

In [53]:
inter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71 entries, 0 to 70
Data columns (total 27 columns):
Player      71 non-null object
MP          71 non-null float64
FG          71 non-null float64
FGA         71 non-null float64
FG%         71 non-null float64
3P          71 non-null float64
3PA         71 non-null float64
3P%         71 non-null float64
2P          71 non-null float64
2PA         71 non-null float64
2P%         71 non-null float64
FT          71 non-null float64
FTA         71 non-null float64
FT%         71 non-null float64
ORB         71 non-null float64
DRB         71 non-null float64
TRB         71 non-null float64
AST         71 non-null float64
STL         71 non-null float64
BLK         71 non-null float64
TOV         71 non-null float64
PF          71 non-null float64
PTS         71 non-null float64
eFGP        71 non-null float64
PER         71 non-null float64
WS          71 non-null float64
Position    29 non-null object
dtypes: float64(25), object(2)
memo

In [55]:
inter.to_csv('processed_inter.csv', index = True)